In [1]:
from pathlib import Path
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from backend.main import generate_video, generate_script, generate_paper, generate_assets #, publish_post_command
from dataclasses import dataclass

# For notebook usage we need to apply the nest_asyncio patch
# !pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

/Users/davidperso/projects/repository_folder/arxflix/env_arxflix/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dotenv
import os
dotenv.load_dotenv()

# Load environment variables
# API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
# GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
# GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

# print(f"API_KEY: {API_KEY}")
# print(f"OPENAI_API_KEY: {OPENAI_API_KEY}")
# print(f"SERPAPI_API_KEY: {SERPAPI_API_KEY}")
# print(f"GOOGLE_CLOUD_PROJECT: {GOOGLE_CLOUD_PROJECT}")
# print(f"GOOGLE_APPLICATION_CREDENTIALS: {GOOGLE_APPLICATION_CREDENTIALS}")


In [3]:
root_path = Path("./pipeline")
root_path

PosixPath('pipeline')

In [4]:
paper_id = "2404.02905"

In [5]:
paper_root = root_path / paper_id
paper_root.mkdir(parents=True,exist_ok=True)

In [6]:
paper_md_target = (paper_root / "paper").with_suffix(".md")
paper_md = generate_paper("arxiv_html",paper_id)
paper_md_target.write_text(f"This is the only paper id, don't use another one :  {paper_id} \n\n"+paper_md)

44414

In [7]:
print(paper_md)



Visual Autoregressive Modeling:  Scalable Image Generation via Next\-Scale Prediction


###### Abstract

We present Visual AutoRegressive modeling (VAR), a new generation paradigm that redefines the autoregressive learning on images as coarse\-to\-fine “next\-scale prediction” or “next\-resolution prediction”, diverging from the standard raster\-scan “next\-token prediction”.
This simple, intuitive methodology allows autoregressive (AR) transformers to learn visual distributions fast and can generalize well: VAR, for the first time, makes GPT\-style AR models surpass diffusion transformers in image generation.
On ImageNet 256$\\times$256 benchmark, VAR significantly improve AR baseline by improving Fréchet inception distance (FID) from 18\.65 to 1\.80, inception score (IS) from 80\.4 to 356\.4, with 20$\\times$ faster inference speed.
It is also empirically verified that VAR outperforms the Diffusion Transformer (DiT) in multiple dimensions including image quality, inference speed, da

In [8]:

# import re
# def replace_keys_with_values(text, dict_list):
#   """
#   Replaces keys found in a text with their corresponding values from a list of dictionaries.

#   Args:
#     text: The input text string.
#     dict_list: A list of dictionaries where keys are patterns to search for in the text 
#                and values are the replacements.

#   Returns:
#     The modified text with keys replaced by values.
#   """

#   # Combine all dictionaries into a single dictionary for efficiency
#   combined_dict = {}
#   for d in dict_list:
#     combined_dict.update(d)

#   # Sort keys by length in descending order to handle overlapping keys correctly
#   sorted_keys = sorted(combined_dict.keys(), key=len, reverse=True)

#   # Build a regular expression pattern to match any of the keys
#   # Escape special characters in keys for use in regex
#   pattern = re.compile("|".join(map(re.escape, sorted_keys)))

#   # Perform the replacement using re.sub with a lambda function
#   modified_text = pattern.sub(lambda match: combined_dict[match.group(0)], text)

#   return modified_text

In [9]:
# print(replace_keys_with_values(paper_md,links))


In [10]:
# print(paper_md)

In [11]:
# def get_link(link,paper_id):
    
#     if 'ar5iv.labs.arxiv.org' in link:
#         return '![]('+link.replace('![](','https://').replace(')','')+')'
#     elif f'https:/arxiv.org/html/{paper_id}/'  in link:
#         return '![]('+link.replace('![](',f'https://arxiv.org/html/{paper_id}/').replace(')','')+')'
#     elif '(arxiv.org' in link:
#         return '![]('+link.replace('![](arxiv.org',f'https://arxiv.org/html/{paper_id}').replace(')','')+')'
#     else:
#         return '![]('+link.replace('![](',f'https://arxiv.org/html/{paper_id}/').replace(')','')+')'

# links  = [{line : get_link(line,paper_id)} for line in paper_md.split('\n') if '![](' in line]


# links


In [12]:
f"The id of the paper is {paper_id} \n\n"

'The id of the paper is 2404.02905 \n\n'

In [13]:
script_txt_target = (paper_root / "script").with_suffix(".txt")
script_txt = generate_script('gemini',paper_md,paper_id)

Validating script structure for paper_id: 2404.02905
Number input_token : 15660


In [15]:
print(script_txt)


\Headline: Visual Autoregressive Modeling (VAR): A New Paradigm for Image Generation
\Text: Hey Arxflix viewers! Today, we're exploring a groundbreaking paper on Visual Autoregressive Modeling, or VAR, which is changing the game in image generation.  This approach reimagines how autoregressive models handle images, leading to significant improvements.
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/fig/abs.jpg
\Text: Check out these impressive images generated by VAR. Notice the detail and quality!
\Headline: From Next-Token to Next-Scale Prediction
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x1.png
\Text: Traditional autoregressive models process images like a raster scan, pixel by pixel.  VAR, however, takes a multi-scale approach, predicting the next resolution level, much like how we perceive images – from coarse to fine detail.
\Headline: Why VAR Outperforms Diffusion Models
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x2.png
\Text:

In [1]:
script_txt = r"""\Headline: Visual Autoregressive Modeling (VAR): A New Paradigm for Image Generation
\Text: Hey Arxflix viewers! Today, we're exploring a groundbreaking paper on Visual Autoregressive Modeling, or VAR, which is changing the game in image generation.  This approach reimagines how autoregressive models handle images, leading to significant improvements.
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/fig/abs.jpg
\Text: Check out these impressive images generated by VAR. Notice the detail and quality!
\Headline: From Next-Token to Next-Scale Prediction
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x1.png
\Text: Traditional autoregressive models process images like a raster scan, pixel by pixel.  VAR, however, takes a multi-scale approach, predicting the next resolution level, much like how we perceive images – from coarse to fine details.
\Text: This simple change dramatically improves efficiency and performance.
\Headline: Outperforming Diffusion Models
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x2.png
\Text: The results are astonishing! VAR achieves a Fréchet Inception Distance (FID) of 1.80 and an Inception Score (IS) of 356.4 on ImageNet 256x256, surpassing leading diffusion models like Diffusion Transformer (DiT). It's also 20 times faster!
\Headline: Scaling Laws and Zero-Shot Capabilities
\Text: The researchers also found that VAR exhibits power-law scaling, similar to large language models. This means performance improves predictably as model size increases.  Plus, VAR shows impressive zero-shot generalization to tasks like image inpainting and editing.
\Figure: https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x3.png
\Text: This figure illustrates the two-stage training process of VAR. First, a multi-scale VQVAE encodes the image into multiple token maps. Then, the VAR transformer predicts the next higher-resolution token map, conditioned on the previous ones.
\Headline: Conclusion: A New Era for Visual Autoregressive Models
\Text: VAR is a significant advancement in image generation. Its multi-scale approach, superior performance, and scaling laws make it a game-changer.  This research opens exciting new avenues for visual AI and its integration with other modalities.
"""

In [14]:
total_duration = generate_assets(
    script=script_txt,
    method="lmnt",
    mp3_output=(paper_root / "audio").with_suffix(".wav").absolute().as_posix(),
    srt_output=(paper_root / "subtitles").with_suffix(".srt").absolute().as_posix(),
    rich_output=(paper_root / "rich").with_suffix(".json").absolute().as_posix(),
)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x148b096d0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x14881e750>, 53225.6749355)])']
connector: <aiohttp.connector.TCPConnector object at 0x13ed61940>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x148b08980>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x14881ddf0>, 53226.423836666)])']
connector: <aiohttp.connector.TCPConnector object at 0x148ab4860>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x14897b860>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x14881ddf0>, 53227.411798583)])']
connector: <aiohttp.connector.TCPConnector object at 0x14897a6c0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x148912270>
Unclosed connector
connections: ['deque([(<aiohttp.client

In [17]:
output_video = (paper_root / "video").with_suffix(".mp4")
generate_video(
    input_dir = paper_root,
    output_video = output_video,
)

Free port: 51066
Exposed directory pipeline/2404.02905


Progress: resolved 1, reused 0, downloaded 0, added 0
Packages: +47
+++++++++++++++++++++++++++++++++++++++++++++++
Progress: resolved 47, reused 42, downloaded 5, added 47, done


Starting up http-server, serving /Users/davidperso/projects/repository_folder/arxflix/pipeline/2404.02905

http-server version: 14.1.1

http-server settings: 
CORS: true
Cache: 3600 seconds
Connection Timeout: 120 seconds
Directory Listings: visible
AutoIndex: visible
Serve GZIP Files: false
Serve Brotli Files: false
Default File Extension: none

Available on:
  http://localhost:51066
Hit CTRL-C to stop the server



(node:23604) ExperimentalWarning: CommonJS module /opt/homebrew/lib/node_modules/npm/node_modules/debug/src/node.js is loading ES Module /opt/homebrew/lib/node_modules/npm/node_modules/supports-color/index.js using require().
Support for loading ES Module in require() is an experimental feature and might change at any time
(Use `node --trace-warnings ...` to show where the warning was created)


Bundling 6%
Bundling 18%
Bundling 54%
Bundling 61%
Bundling 69%
Bundling 74%
Bundling 80%
Bundling 85%
Bundling 90%
Bundling 95%
Bundling 100%
Getting compositions
[Sat Dec 07 2024 16:43:46 GMT+0100 (Central European Standard Time)]  "GET /audio.wav" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.6778.109 Safari/537.36"
Composition        Arxflix
Codec              h264
Output             /Users/davidperso/projects/repository_folder/arxflix/pipeline/2404.02905/video.mp4
Concurrency        4x
Rendered 0/3987
[Sat Dec 07 2024 16:43:47 GMT+0100 (Central European Standard Time)]  "GET /subtitles.srt" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.6778.109 Safari/537.36"
[Sat Dec 07 2024 16:43:47 GMT+0100 (Central European Standard Time)]  "GET /rich.json" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.6778.

...

Rendered 61/3987, time remaining: 7m 21s
[Sat Dec 07 2024 16:43:48 GMT+0100 (Central European Standard Time)]  "GET /audio.wav" "Mozilla/5.0 (@remotion/renderer - https://remotion.dev)"
Rendered 62/3987, time remaining: 8m 35s
Rendered 63/3987, time remaining: 8m 50s
Rendered 64/3987, time remaining: 8m 10s
Rendered 65/3987, time remaining: 8m 7s
Rendered 66/3987, time remaining: 8m 4s
Rendered 67/3987, time remaining: 8m 1s
Rendered 68/3987, time remaining: 8m 58s
Rendered 69/3987, time remaining: 8m 54s
Rendered 70/3987, time remaining: 8m 51s
Rendered 71/3987, time remaining: 8m 47s
Rendered 72/3987, time remaining: 8m 44s
Rendered 73/3987, time remaining: 8m 41s
Rendered 74/3987, time remaining: 8m 38s
Rendered 75/3987, time remaining: 8m 36s
Rendered 76/3987, time remaining: 8m 34s
Rendered 77/3987, time remaining: 8m 31s
Rendered 78/3987, time remaining: 7m 28s
Rendered 79/3987, time remaining: 7m 26s
Rendered 80/3987, time remaining: 7m 23s
Rendered 81/3987, time remaining: 7m 2

 console.error  Access to fetch at 'https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png' from origin 'http://localhost:3000' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource. If an opaque response serves your needs, set the request's mode to 'no-cors' to fetch the resource with CORS disabled.
Browser failed to load https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png (Fetch): net::ERR_FAILED
 https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png  Failed to load resource: net::ERR_FAILED
 console.error  Access to fetch at 'https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png' from origin 'http://localhost:3000' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource. If an opaque response serves your needs, set the request's mode to 'no-cors' to fetch the resource with CORS disabled.
Browser failed to load https://ar5iv.labs.arxiv.org//html/2404.

Rendered 87/3987, time remaining: 7m 9s
Rendered 88/3987, time remaining: 7m 6s
Rendered 89/3987, time remaining: 7m 5s
Rendered 90/3987, time remaining: 7m 3s
Rendered 91/3987, time remaining: 7m 1s
Rendered 92/3987, time remaining: 7m 59s
Rendered 93/3987, time remaining: 7m 57s
Rendered 94/3987, time remaining: 7m 55s
Rendered 95/3987, time remaining: 7m 54s
Rendered 96/3987, time remaining: 7m 51s
Rendered 97/3987, time remaining: 7m 50s
Rendered 98/3987, time remaining: 7m 48s


 console.error  Access to fetch at 'https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png' from origin 'http://localhost:3000' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource. If an opaque response serves your needs, set the request's mode to 'no-cors' to fetch the resource with CORS disabled.
Browser failed to load https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png (Fetch): net::ERR_FAILED
 https://ar5iv.labs.arxiv.org//html/2404.02905/assets/x6.png  Failed to load resource: net::ERR_FAILED


Rendered 99/3987, time remaining: 7m 47s
Rendered 100/3987, time remaining: 7m 46s
Rendered 101/3987, time remaining: 7m 44s
Rendered 102/3987, time remaining: 7m 43s
Rendered 103/3987, time remaining: 7m 41s
Rendered 104/3987, time remaining: 7m 39s
Rendered 105/3987, time remaining: 7m 38s
Rendered 106/3987, time remaining: 7m 37s
Rendered 107/3987, time remaining: 7m 35s
Rendered 108/3987, time remaining: 7m 34s
Rendered 109/3987, time remaining: 7m 32s
Rendered 110/3987, time remaining: 7m 31s
Rendered 111/3987, time remaining: 6m 29s
Rendered 112/3987, time remaining: 6m 28s
Rendered 113/3987, time remaining: 6m 27s
Rendered 114/3987, time remaining: 6m 26s
Rendered 115/3987, time remaining: 6m 25s
Rendered 116/3987, time remaining: 6m 24s
Rendered 117/3987, time remaining: 6m 23s
Rendered 118/3987, time remaining: 6m 22s
Rendered 119/3987, time remaining: 6m 21s
Rendered 120/3987, time remaining: 6m 20s
Rendered 121/3987, time remaining: 6m 19s
Rendered 122/3987, time remaining: 

In [ ]:

# def query_llm_studio2(symstem_prompt,prompt, pydantic_model,model_name="default"):  # Use "default" or your model name
#     """
#     Queries the LLM Studio model.

#     Args:
#         prompt (str): The text prompt to send to the model.
#         model_name (str): The name of the model (default is usually "default").

#     Returns:
#         str: The model's response, or None if an error occurs.
#     """
#     try:
#             messages=[
#                 {"role": "system", "content": symstem_prompt}
#                 ,{"role": "user", "content": prompt}]


#             completion, raw = client_openai.chat.create_with_completion(
#             model=model_name,
#             messages=messages,
#             response_model=pydantic_model,
#             # generation_config={
#             #     "temperature": 0.0,
#             #     "top_p": 1,
#             #     "candidate_count": 1,
#             #     "max_output_tokens": 8000,
#             # },
#             max_retries=10,
#         )
#             result = completion.model_dump()
#             # Assuming the response structure is the same as the OpenAI API
#             return result
#     except Exception as e:
#         print(f"Error querying LLM Studio: {e}")
#         return None
# # Example usage:

# import json
# from models.ai_responce import SimilarIDS



# #load json data
# with open("data.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

from llm_agent.querry_model import get_list_10_similar

# trying other providers for audio


In [ ]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq()


In [ ]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq()

# Specify the path to the audio file
filename = "/var/folders/3j/jv5_hbgn59g9yxccxtfcvz4r0000gp/T/audio_1.wav" # Replace with your audio file!

# Open the audio file
with open(filename, "rb") as file:
    # Create a translation of the audio file
    translation = client.audio.transcriptions.create(
      language="en",
      file=(filename, file.read()), # Required audio file
      model="whisper-large-v3", # Required model to use for translation
      # prompt="Specify context or spelling",  # Optional
      response_format="verbose_json",  # Use verbose_json for timestamps
      temperature=0.0,  # Optional
      
    )
    # Print the translation text
    print(translation)

In [ ]:
translation.segments

In [ ]:
import mlx_whisper
audio_path = "/var/folders/3j/jv5_hbgn59g9yxccxtfcvz4r0000gp/T/audio_1.wav"
result = mlx_whisper.transcribe(audio=audio_path,word_timestamps=True)



In [ ]:
float(result["segments"][-1]["end"]-result["segments"][0]["start"])



In [ ]:
from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
    FileSource,
)

audio_path = "/var/folders/3j/jv5_hbgn59g9yxccxtfcvz4r0000gp/T/audio_1.wav"
deepgram = DeepgramClient()
with open(audio_path, "rb") as file:
    buffer_data = file.read()

payload: FileSource = {
    "buffer": buffer_data,
}

#STEP 2: Configure Deepgram options for audio analysis
options = PrerecordedOptions(
    model="nova-2",
    smart_format=True,
)

# STEP 3: Call the transcribe_file method with the text payload and options
response = deepgram.listen.rest.v("1").transcribe_file(payload, options)

In [ ]:
result= response.to_dict()["results"]["channels"][0]["alternatives"][0]["words"]
result[-1]["end"]-result[0]["start"]